In [1]:
import polars as pl
import os

In [2]:
pl.Config(fmt_str_lengths=1000)

In [3]:
files = os.listdir('data/domain')

In [4]:
tebo = os.listdir('data/domain')

In [5]:
for parquet in tebo:
    if parquet.endswith('.parquet'):
        print(parquet)

estabelecimentos0_silver.parquet
estabelecimentos1_silver.parquet
estabelecimentos2_silver.parquet
estabelecimentos3_silver.parquet
estabelecimentos4_silver.parquet
estabelecimentos5_silver.parquet
estabelecimentos6_silver.parquet
estabelecimentos7_silver.parquet
estabelecimentos8_silver.parquet
estabelecimentos9_silver.parquet


In [6]:
df_emp = pl.read_parquet(f'data/domain/{files[2]}')

In [7]:
df_emp.head(6)

cnpj_completo,identificador_matriz,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,pais,data_inicio_atividade,cnae_principal,cnae_secundario,numero,complemento,bairro,cep,uf,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_fax,fax,email,situacao_especial,data_situacao_especial,logradouro_completo
str,i8,str,i8,date,i16,str,i32,date,i64,list[str],str,str,str,str,cat,i32,str,str,str,str,str,str,str,cat,date,str
"""27875213000113""",1,null,2,2024-01-18,0,"""""",null,2017-06-01,9001906,"[""""]","""120""","""BLOCO I;APT 03""","""BAETA NEVES""","""09760410""","""SP""",7075,"""11""","""23747104""",null,null,"""""","""""",null,"""""",null,"""RUA OSEAS DE PAULA CAMPOS"""
"""26244670000570""",2,"""ROTA SERVICE ILHA DO GOVERNADOR""",8,2023-02-24,1,"""""",null,2019-11-13,4541203,"[""4520007,4541204,4541206,4781400""]","""00529""",""" LOJ A""","""JARDIM GUANABARA""","""21931383""","""RJ""",6001,"""21""","""40404033""",null,null,"""""","""""",null,"""""",null,"""ESTRADA DO GALEAO"""
"""35509654000120""",1,null,2,2019-11-13,0,"""""",null,2019-11-13,5611201,"[""5611203""]","""510""","""LETRA A""","""PEDRINHA""","""12508010""","""SP""",6469,"""12""","""31276182""",null,null,"""""","""""","""LCBRJ@TERRA.COM.BR""","""""",null,"""ESTRADA PRESIDENTE TANCREDO NEVES"""
"""35510044000146""",1,null,2,2023-01-31,0,"""""",null,2019-11-13,4772500,"[""""]","""89""",null,"""JARDIM PLANALTO""","""59155256""","""RN""",1779,"""84""","""98566127""",null,null,"""""","""""","""IRLAFRANCELE@HOTMAIL.COM""","""""",null,"""RUA CONCORDE"""
"""35510441000118""",1,"""BAR DO JONAS""",8,2019-11-13,1,"""""",null,2019-11-13,5611204,"[""4729601,4723700,5611201,5611203""]","""1100""","""LOJA""","""MARIA ORTIZ""","""29070350""","""ES""",5705,"""27""","""99622587""",null,null,"""""","""""","""MARCELO.NET77@HOTMAIL.COM""","""""",null,"""AVENIDA JERONIMO VERVLOET"""
"""35510848000145""",1,null,2,2019-11-13,0,"""""",null,2019-11-13,1052000,"[""5612100""]","""3351""","""CASA;CASA;CASA;CASA""","""CONSELHEIRO PAULINO""","""28635010""","""RJ""",5867,"""22""","""98231774""",null,null,"""""","""""","""JAIMEJUNIOR610@GMAIL.COM""","""""",null,"""AVENIDA NOSSA SENHORA DO AMPARO"""


In [8]:
#Cleaning ddd's fields

In [8]:
df_emp = df_emp.with_columns(
    pl.when(~pl.col('ddd_1').str.contains("-?\d+(?:\.\d+)?")).then(pl.lit(None)).otherwise(pl.col('ddd_1')).alias('ddd1_alias').str.strip_chars().cast(pl.Int32),
    pl.when(~pl.col('ddd_2').str.contains("-?\d+(?:\.\d+)?")).then(pl.lit(None)).otherwise(pl.col('ddd_2')).alias('ddd2_alias').str.strip_chars().cast(pl.Int32)
).drop(['ddd_1','ddd_2'])

In [9]:
### change to matriz / filial

In [10]:
df_emp = df_emp.with_columns(
    pl.col('identificador_matriz').replace([1,2],['matriz','filial']).cast(pl.Categorical)
)

##### Support table - motivo

In [11]:
support_motivo_path = os.listdir('data/raw/support/motivos/')[0]

In [12]:
motivo_column = [
    'situacao_cadastral',
    'descricao_motivo'
]

In [13]:
dtypes_motivo = {
    'situacao_cadastral':pl.Int8,
    'descricao_motivo':pl.Categorical
}

In [14]:
df_mot = pl.read_csv(f'data/raw/support/motivos/{support_motivo_path}', separator = ';', new_columns = motivo_column,dtypes=dtypes_motivo)

##### Support table - municipios

In [15]:
support_municipio_path = os.listdir('data/raw/support/municipios/')[0]

In [16]:
municipio_column = [
    'municipio',
    'descricao_municipio'
]

In [17]:
dtypes_municipio = {
    'municipio':pl.Int32
}

In [18]:
df_mun = pl.read_csv(f'data/raw/support/municipios/{support_municipio_path}',separator = ';',new_columns = municipio_column, dtypes=dtypes_municipio)

##### Support table - pais

In [19]:
support_pais_path = os.listdir('data/raw/support/paises/')[0]

In [20]:
pais_column = [
    'pais',
    'descricao_pais'
]

In [21]:
dtypes_pais = {
    'pais':pl.Int32
}

In [22]:
df_paises = pl.read_csv(f'data/raw/support/paises/{support_pais_path}',separator = ';',encoding='latin1', new_columns = pais_column, dtypes=dtypes_pais)

##### Joining data

In [23]:
#sit. cadastral
df_emp = df_emp.join(df_mot, on="situacao_cadastral",how='left')

In [24]:
#municipio
df_emp = df_emp.join(df_mun, on="municipio",how='left')

In [25]:
#paises
df_emp = df_emp.join(df_paises, on="pais",how='left')

In [26]:
select_columns = [
    'cnpj_completo',
    'nome_fantasia',
    'identificador_matriz',
    'data_inicio_atividade',
    'descricao_pais',
    'uf',
    'descricao_municipio',
    'cep',
    'logradouro_completo',
    'bairro',
    'numero',
    'complemento',
    'ddd1_alias',
    'telefone_1',
    'ddd2_alias',
    'telefone_2',
    'email',
    'cnae_principal',
    'descricao_motivo'
]

In [27]:
df_new = df_emp.select(select_columns)

In [30]:
df_new.filter(pl.col('cep').is_null())

cnpj_completo,nome_fantasia,identificador_matriz,data_inicio_atividade,descricao_pais,uf,descricao_municipio,cep,logradouro_completo,bairro,numero,complemento,ddd1_alias,telefone_1,ddd2_alias,telefone_2,email,cnae_principal,descricao_motivo
str,str,cat,date,str,cat,str,str,str,str,str,str,i32,str,i32,str,str,i64,cat
"""35645008000190""",null,"""matriz""",2019-11-13,"""ITALIA""","""EX""","""EXTERIOR""",null,""" VIA SAN DONATO""","""CAP 40127""","""191""",null,null,null,null,null,"""MARCELAGUIMARAES2002@YAHOO.COM.BR""",6463800,"""INCORPORACAO"""
"""35828810000115""","""SJRD""","""matriz""",2019-12-19,"""ESTADOS UNIDOS""","""EX""","""EXTERIOR""",null,""" 251 LITTLE FALLS DRIVE""","""WILMINGTON""","""S/N""","""DELAWARE 19808""",null,null,null,null,"""CNPJ@ROMERORODRIGUES.COM.BR""",6462000,"""INCORPORACAO"""
"""36102240000144""","""ACERTA S.L.""","""matriz""",2020-01-22,"""ESPANHA""","""EX""","""EXTERIOR""",null,""" CALLE LOPEZ DE HOYOS, 35""","""MADRI""","""S/N""",null,null,null,null,null,"""ARAMALLO@VVRLAW.COM""",7111100,"""INCORPORACAO"""
"""36184438000114""","""LIV ECO HABITATS""","""matriz""",2020-01-29,"""ESTADOS UNIDOS""","""EX""","""EXTERIOR""",null,""" 7901 4TH ST N STE 300""","""ST. PETERSBURG""","""S/N""","""33702""",null,null,null,null,"""FFT1@YMAIL.COM""",6462000,"""INCORPORACAO"""
"""36201164000124""","""CUPH""","""matriz""",2020-01-30,"""Países Baixos (Holanda)""","""EX""","""EXTERIOR""",null,""" BOOMPJES 270""","""ROTERDAN""","""S/N""","""3011 XZ""",null,null,null,null,"""FISCAL@CONTROLUNION.COM.BR""",6462000,"""INCORPORACAO"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""27866690000112""","""TRIBLUE""","""matriz""",2017-05-31,"""URUGUAI""","""EX""","""EXTERIOR""",null,""" AVENIDA 18 DE JULIO, 878""","""MONTEVIDEO""","""S/N""","""OFICINA 1204""",null,null,null,null,null,4751201,"""INCORPORACAO"""
"""27877079000190""",null,"""matriz""",2017-06-01,"""COREIA REPUBLICA DA""","""EX""","""EXTERIOR""",null,""" HSBC BUILDING,37 CHILPAE-RO, JUNG-GU""","""SEOUL""",""" 0""",null,null,null,null,null,null,6630400,"""FUSAO"""
"""27877546000181""","""SKY AIRCRAFT B3""","""matriz""",2017-06-01,"""IRLANDA""","""EX""","""EXTERIOR""",null,""" 70 SIR JOHN ROGERSON S QUAY""","""DUBLIN""","""S/N""",null,null,null,null,null,"""BASRAM@BASCHRAMEH.COM.BR""",6440900,"""INCORPORACAO"""


In [28]:
df_new.filter(pl.col('cep').str.len_chars() < 7)

cnpj_completo,nome_fantasia,identificador_matriz,data_inicio_atividade,descricao_pais,uf,descricao_municipio,cep,logradouro_completo,bairro,numero,complemento,ddd1_alias,telefone_1,ddd2_alias,telefone_2,email,cnae_principal,descricao_motivo
str,str,cat,date,str,cat,str,str,str,str,str,str,i32,str,i32,str,str,i64,cat


In [29]:
df_emp.select(select_columns).filter(pl.col('identificador_matriz')=='matriz')

cnpj_completo,nome_fantasia,identificador_matriz,data_inicio_atividade,descricao_pais,uf,descricao_municipio,cep,logradouro_completo,bairro,numero,complemento,ddd1_alias,telefone_1,ddd2_alias,telefone_2,email,cnae_principal,descricao_motivo
str,str,cat,date,str,cat,str,str,str,str,str,str,i32,str,i32,str,str,i64,cat
"""27875213000113""",null,"""matriz""",2017-06-01,null,"""SP""","""SAO BERNARDO DO CAMPO""","""09760410""","""RUA OSEAS DE PAULA CAMPOS""","""BAETA NEVES""","""120""","""BLOCO I;APT 03""",11,"""23747104""",null,null,null,9001906,"""INCORPORACAO"""
"""35509654000120""",null,"""matriz""",2019-11-13,null,"""SP""","""GUARATINGUETA""","""12508010""","""ESTRADA PRESIDENTE TANCREDO NEVES""","""PEDRINHA""","""510""","""LETRA A""",12,"""31276182""",null,null,"""LCBRJ@TERRA.COM.BR""",5611201,"""INCORPORACAO"""
"""35510044000146""",null,"""matriz""",2019-11-13,null,"""RN""","""PARNAMIRIM""","""59155256""","""RUA CONCORDE""","""JARDIM PLANALTO""","""89""",null,84,"""98566127""",null,null,"""IRLAFRANCELE@HOTMAIL.COM""",4772500,"""INCORPORACAO"""
"""35510441000118""","""BAR DO JONAS""","""matriz""",2019-11-13,null,"""ES""","""VITORIA""","""29070350""","""AVENIDA JERONIMO VERVLOET""","""MARIA ORTIZ""","""1100""","""LOJA""",27,"""99622587""",null,null,"""MARCELO.NET77@HOTMAIL.COM""",5611204,"""TRANSPASSE"""
"""35510848000145""",null,"""matriz""",2019-11-13,null,"""RJ""","""NOVA FRIBURGO""","""28635010""","""AVENIDA NOSSA SENHORA DO AMPARO""","""CONSELHEIRO PAULINO""","""3351""","""CASA;CASA;CASA;CASA""",22,"""98231774""",null,null,"""JAIMEJUNIOR610@GMAIL.COM""",1052000,"""INCORPORACAO"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""27893349000156""",null,"""matriz""",2017-06-05,null,"""RJ""","""PARATY""","""23970000""","""RUA Tangara""","""Cabore""","""01""","""APT 01C""",35,"""33713812""",null,null,"""davog.byrne@gmail.com""",8599603,"""INCORPORACAO"""
"""27893365000149""","""ELIANA FERNANDES""","""matriz""",2017-06-05,null,"""BA""","""SALVADOR""","""41820020""","""AVENIDA TANCREDO NEVES""","""CAMINHO DAS ARVORES""","""274""",""" CENTRO EMP. IGUATEMI BLOCO B SALA 734 E 732""",71,"""99792335""",null,null,"""ELIANAFERNANDESMRV@GMAIL.COM""",6821801,"""CISAO TOTAL"""
"""27893378000118""",null,"""matriz""",2017-06-05,null,"""PI""","""CURRAIS""","""64905000""","""COMUNIDADE BREJO DA CONCEICAO""","""ZONA RURAL""","""S/N""",null,86,"""81839932""",null,null,"""LUZIENEDEFRACA092@GMAIL.COM""",4712100,"""INCORPORACAO"""


In [30]:
df_new['complemento'].value_counts(sort=True)

complemento,count
str,u32
null,2799702
"""CASA""",206541
"""TERREO""",37796
"""CONJ""",33671
"""LOJA""",33578
…,…
"""BLOCO 07 APTO 101 - RESIDENCIAL JARDIM COSTA VERDE""",1
"""ARMZ 15 E 16""",1
"""QUADRA16 LOTE 16-17""",1
